In [1]:
from tensor2tensor.data_generators import generator_utils
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

import tensorflow as tf


import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
from tensorflow.contrib.eager.python import tfe
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("./data")
tmp_dir = os.path.expanduser("./tmp")
train_dir = os.path.expanduser("./train")
checkpoint_dir = os.path.expanduser("./checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data"
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"


/Users/boyalex/shad/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('he-en/en.train.txt') as f, open('he-en/he.train.txt') as g, open('en_he.train.txt', 'w') as out:
    for en, he in zip(f, g):
        out.write(en.strip() + '\t' + he.strip() + '\n')
        
with open('he-en/en.dev.txt') as f, open('he-en/he.dev.txt') as g, open('en_he.dev.txt', 'w') as out:
    for en, he in zip(f, g):
        out.write(en.strip() + '\t' + he.strip() + '\n')
        
with open('he-en/en.test.txt') as f, open('he-en/he.test.txt') as g, open('en_he.test.txt', 'w') as out:
    for en, he in zip(f, g):
        out.write(en.strip() + '\t' + he.strip() + '\n')

In [3]:
all_data = []

with open('en_he.train.txt', 'r') as f:
    all_data += [l.strip() for l in f]
with open('en_he.dev.txt', 'r') as f:
    all_data += [l.strip() for l in f]
with open('en_he.test.txt', 'r') as f:
    all_data += [l.strip() for l in f]

with open('main_dataset.txt', 'w') as f:
    f.write('\n'.join(all_data))

In [4]:
!head main_dataset.txt

bimdurot	במדורות
sheyikrom	שיקרום
iter	איטר
toses	תוסס
kvoe	קב
bshilya	בשליא
liftokh	לפתוח
shvutchem	שבותכם
mtse	מצה
mtserin	מצרין


In [5]:
@registry.register_problem
class HEENTranslitProblem(text_problems.Text2TextProblem):
    filename = './main_dataset.txt'
    name = 'heen_translit'
    
    @property
    def is_generate_per_split(self):
        return True
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        """Generate samples of input text and target text pairs.
        Each yielded dict will be made into a single example. The values should be
        raw text. The Problem will generate a vocabulary and encode the raw text as
        integers as part of the data generation process.
        This method is typically called once per split in `self.dataset_splits`
        unless `self.is_generate_per_split=False`.
        Args:
          data_dir: final data directory. Typically only used in this method to copy
            over user-supplied vocab files (for example, if vocab_type ==
            VocabType.TOKEN).
          tmp_dir: temporary directory that you can use for downloading and scratch.
          dataset_split: problem.DatasetSplit, which data split to generate samples
            for (for example, training and evaluation).
        Yields:
          {"inputs": text, "targets": text}
        """
        with open(self.filename) as f:
            for l in f:
                en, he = l.strip().split('\t')
                yield {
                    "inputs": he,
                    "targets": en
                }
#     raise NotImplementedError()

    @property
    def vocab_type(self):
        """What kind of vocabulary to use.
        `VocabType`s:
          * `SUBWORD`: `SubwordTextEncoder`, an invertible wordpiece vocabulary.
            Must provide `self.approx_vocab_size`. Generates the vocabulary based on
            the training data. To limit the number of samples the vocab generation
            looks at, override `self.max_samples_for_vocab`. Recommended and
            default.
          * `CHARACTER`: `ByteTextEncoder`, encode raw bytes.
          * `TOKEN`: `TokenTextEncoder`, vocabulary based on a file. Must provide a
            vocabulary file yourself (`TokenTextEncoder.store_to_file`) because one
            will not be generated for you. The vocab file should be stored in
            `data_dir/` with the name specified by `self.vocab_filename`.
        Returns:
          VocabType constant
        """
        return text_problems.VocabType.CHARACTER

In [6]:
from tensor2tensor.utils import trainer_lib
# data_dir = '.'


model_name = "lstm_seq2seq_attention"
hparams_set = "lstm_attention"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="heen_translit_problem")
print (hparams.max_length)
hparams.set_hparam('max_length', 12)
print (hparams.max_length)

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_model = registry.model(model_name)(hparams, Modes.EVAL)

0
12
INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-03-13 22:49:47,479] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


[2018-03-13 22:49:47,485] Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0


[2018-03-13 22:49:47,490] Setting hparams.layer_prepostprocess_dropout to 0.0


INFO:tensorflow:Setting hparams.symbol_dropout to 0.0


[2018-03-13 22:49:47,495] Setting hparams.symbol_dropout to 0.0


In [7]:
problem = HEENTranslitProblem()

In [8]:
problem.generate_data(data_dir, tmp_dir)

INFO:tensorflow:Skipping generator because outputs files exist


[2018-03-13 22:49:47,621] Skipping generator because outputs files exist


INFO:tensorflow:Skipping generator because outputs files exist


[2018-03-13 22:49:47,629] Skipping generator because outputs files exist


INFO:tensorflow:Skipping shuffle because output files exist


[2018-03-13 22:49:47,649] Skipping shuffle because output files exist


In [9]:
@tfe.implicit_value_and_gradients
def loss_fn(features):
  _, losses = translate_model(features)
  return losses["training"]

# Setup the training data
BATCH_SIZE = 128
train_dataset = problem.dataset(Modes.TRAIN, data_dir)
train_dataset = train_dataset.repeat(None).batch(BATCH_SIZE)

optimizer = tf.train.AdamOptimizer()

INFO:tensorflow:Reading data files from ./data/heen_translit_problem-train*


[2018-03-13 22:49:47,679] Reading data files from ./data/heen_translit_problem-train*


INFO:tensorflow:partition: 0 num_data_files: 100


[2018-03-13 22:49:47,687] partition: 0 num_data_files: 100


In [10]:
NUM_STEPS = 500

for count, example in enumerate(tfe.Iterator(train_dataset)):
  example["targets"] = tf.reshape(example["targets"], [BATCH_SIZE, 1, 1, 1])  # Make it 4D.
  loss, gv = loss_fn(example)
  optimizer.apply_gradients(gv)

  if count % 50 == 0:
    print("Step: %d, Loss: %.3f" % (count, loss.numpy()))
  if count >= NUM_STEPS:
    break

InvalidArgumentError: Cannot batch tensors with different shapes in component 0. First element had shape [11] and element 1 had shape [17]. [Op:IteratorGetNextSync]

In [12]:
# Fetch the problem
# ende_problem = problems.problem("translate_ende_wmt32k")
# ende_problem.generate_data(data_dir, tmp_dir)

# Copy the vocab file locally so we can encode inputs and decode model outputs
# All vocabs are stored on GCS
# vocab_file = os.path.join(gs_data_dir, "vocab.ende.32768")
# !gsutil cp {vocab_file} {data_dir}

# Get the encoders from the problem
encoders = problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers))

In [13]:
# Generate and view the data
# This cell is commented out because WMT data generation can take hours

# ende_problem.generate_data(data_dir, tmp_dir)
example = tfe.Iterator(problem.dataset(Modes.TRAIN, data_dir)).next()
inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.



# Example inputs as int-tensor.
print("Inputs, encoded:")
print(inputs)
print("Inputs, decoded:")
# Example inputs as a sentence.
print(decode(inputs))
# Example targets as int-tensor.
print("Targets, encoded:")
print(targets)
# Example targets as a sentence.
print("Targets, decoded:")
print(decode(targets))

INFO:tensorflow:Reading data files from ./data/heen_translit_problem-train*


[2018-03-13 22:56:31,674] Reading data files from ./data/heen_translit_problem-train*


INFO:tensorflow:partition: 0 num_data_files: 100


[2018-03-13 22:56:31,698] partition: 0 num_data_files: 100


Inputs, encoded:
[217, 151, 217, 150, 217, 149, 217, 166, 217, 150, 1]
Inputs, decoded:
והדפה
Targets, encoded:
[120, 106, 99, 102, 99, 104, 99, 106, 1]
Targets, decoded:
vhadafah


In [ ]:
data_dir

In [ ]:
!ls train

In [ ]:
!ls

In [15]:
problem.dataset()

TypeError: dataset() missing 1 required positional argument: 'mode'

In [ ]:
model.set_mode(Modes.EVAL)
mnist_eval_dataset = mnist_problem.dataset(Modes.EVAL, data_dir)

# Create eval metric accumulators for accuracy (ACC) and accuracy in
# top 5 (ACC_TOP5)
metrics_accum, metrics_result = metrics.create_eager_metrics(
    [metrics.Metrics.ACC, metrics.Metrics.ACC_TOP5])

for count, example in enumerate(tfe.Iterator(mnist_eval_dataset)):
  if count >= 200:
    break

  # Make the inputs and targets 4D
  example["inputs"] = tf.reshape(example["inputs"], [1, 28, 28, 1])
  example["targets"] = tf.reshape(example["targets"], [1, 1, 1, 1])

  # Call the model
  predictions, _ = model(example)

  # Compute and accumulate metrics
  metrics_accum(predictions, example["targets"])

# Print out the averaged metric values on the eval data
for name, val in metrics_result().items():
  print("%s: %.2f" % (name, val))